# Figure 4
This notebook contains three major parts: 
- I. initialize test instances based on instances generated from previous notebook  
  - Note, you must run **Figure 3_step1_BaselineLP Generation** and **Figure 3_step2_(i)_Learn c part I**, before this
- II. Experiment (lean A, b and c jointly for parametric lps using deep_inv_opt package)
- III. Data Analysis (generating boxplot as Figure 4 in our paper)

In [ ]:
# Copyright (C) to Yingcong Tan, Andrew Delong, Daria Terekhov. All Rights Reserved.

import numpy as np
import numpy.random as npr
import torch
import torch.nn.functional
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams['figure.max_open_warning'] = 0
%matplotlib inline
import deep_inv_opt as io
import deep_inv_opt.plot as iop
import random
import time
import scipy.optimize as opt
from scipy.spatial import ConvexHull
import os

# Make the notebook stretch to 100% browser width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))

as_tensor = io.as_tensor
as_numpy = io.as_numpy
as_str = io.as_str
PATH = "C:/Users/Public/Downloads/"

def seed_random(seed=0):
    np.random.seed(seed)
    random.seed(seed)

### Quick check if required data have been generated

#### Note that, you will need to genereate the test instances for figure3(i) since they are used in this notebook to generate PLP instances.
Please make sure that you have done the following two steps before runing this notebook:
- **Figure 3_step1_BaselineLP Generation.ipynb**, *completely*  
- **Figure 3_step2_(i)_Learn c.ipynb**, *Part I*  
The following cell will quickly check ifyou already have all the required data files from figure3(i), if no error raised, you might continue.

In [ ]:
nVar_range = [2,10]
nCons_range = [[8],[36]]
num_lp = 50


for i, nVar in enumerate (nVar_range):
    for nCons in nCons_range[i]:
        for ind_lp in range(num_lp):
            directory = PATH + "Deep_Inv_Opt/figure3_i/n=%d,m=%d/%svar_LP%s.txt"%(nVar, nCons, str(nVar).zfill(2),str(ind_lp+1).zfill(2))
            if not os.path.exists(directory):
                raise RuntimeError("Test instance for Figure3_i does not exist, \n You must run the code for generating Figure 3(i) first, that is: \n\t\"Figure 3_step1_BaselineLP Generation.ipynb\" complete \n\t\"Figure 3_step2_(i)_Learn c.ipynb\" part I, \n because it generates the LP instances that are used in this notebookto define the PLPs")


## Learning Parametric LP 
### Overview
- PLP instance with linear function: param = w0 + w1*u
 - for each entry of c, add param(w1,w2) = w1 + w2*u
 - for each row of A, randomly select one entry to add param(w3,w4) = w3 + w4*u
 - for each entry of b, add param(w5,w6) = w5 + w6*u

- Read c, A and b  from LP instances for Figure 3.(i), and then set to be $c, A, b$  
- add parametric terms, i.e., param(w1=0,w2), param(w3=0,w4) and param(w5=0,w6) to $c, A\;\&\; b$ to obtain $c_{corrupt}, A_{corrupt}, b_{corrupt}$, that is: 
 - $c_{true} = c + w_1 + w_2*u$,  
 - $A_{true} = A + w_3 + w_4*u$,  
 - $b_{true} = b + w_5 + w_6*u$, 
 - where $w_1 = w_3 = w_5 =0$, and $w_2, w_4\; \&\; w_6 \in U[-\sigma, \sigma]$  
- find a set of feasible u value, and then computer $x_{target}$ by solving PLP($c_{true},\; A_{true},\; b_{true},\; u$)
- then add random noise ($U[-\sigma_{corrupt}, \sigma_{corrupt}]$) to every parameter $w_1,...,w_6$ to generate $c_{corrupt},\; A_{corrupt},\; b_{corrupt}$, that is, 
 - $c_{corrupt} = c_{true} + w_1' + w_2' * u$,  
 - $A_{corrupt} = A_{true} + w_3' + w_4' * u$,      
 - $b_{corrupt} = b_{true} + w_5' + w_6' * u$,  
 - where, $w_1' = w_1 + noise$, ..., $w_6' = w_6 + noise$
- Starting from $c_{corrupt},\; A_{corrupt},\; b_{corrupt}$, learng parameters $w_1', ..., w_6'$ using Deep_Inv_Opt package and attempt to recover the PLP so that error function is minimized (e.g., $\min MSE(x_{predict},\;x_{true})$ )


### Utility Functions
- **Read_3aLP**: read data (A_ub, b_ub, c_init) from instances in figure 3(i)
- **solve_lp**: run linprog function in the Deep_Inv_Opt package to solve a lp
- **feasibility**: checking feasibility
- **record_PLP**: record data of the parametric lps
 - *Problem size*, m,n = A.shape
 - *Base_LP*: c, A_ub, b_ub = LP, from 3a
 - *True Weights*: weights initialized for the true PLP, [wo, w1], where w0 =0, w1 = normal(0,sigma)
 - *w_A_coefficient*:  for w_A, also store a random coefficient (0 or 1 for 2d case) which indicates where to insert the parametric term of w0+w1*u
 - *u_data*: lowerbound (u_lb), upperbound(u_ub) and u_train (equaliy spaced between u_lb and u_ub)
 - *Target points*: x_target, given true_weights, generate corresponding true_PLP, and solve through linprog to obtain x_target.
 - *weights_corrupt*: corrupt the initial weights by adding noise $U(-\sigma, \sigma)$  
- **read_PLP**: Read data from PLP file


In [ ]:
def Read_3aLP(filename):           
    lines=[]    
    with open(filename, "rt") as fp:
        lines=fp.readlines()
        for line in fp:
            lines.append(line.rstrip('\n')) 
    
    m,n = np.fromstring(lines[0], dtype=int, sep=' ')
    
    temp = [lines[i+1: i+1+m] for i in range(len(lines)) if "A_ub" in lines[i]]
    A_ub=np.zeros((m,n))
    b_ub=np.zeros((m,1))
    for j in range(m):
        A_ub[j] = np.fromstring(temp[0][j], dtype=float, sep=' ')
    
    temp = [lines[i+1: i+1+m] for i in range(len(lines)) if "b_ub" in lines[i]]
    for j in range(m):
        b_ub[j] = np.fromstring(temp[0][j], dtype=float, sep=' ')
        
    temp = [lines[j+1] for j in range(len(lines)) if "c_init" in lines[j]]
    c_init = np.fromstring(temp[0], dtype=float, sep=' ') 

    temp = [lines[j+1] for j in range(len(lines)) if "vertex target" in lines[j]]
    x_target = np.fromstring(temp[0], dtype=float, sep=' ') 

    return (A_ub, b_ub, x_target, c_init)

# given c, A and b, solve foward problem once using linprog to check the LP feasibility        
def solve_lp(c, A,b):
    # -1<= c <=1, domain of values for c 
    # normalize the vector so that ||c||_1 = 1
    m, n = A.shape    
    c_vector = io.tensor(c).view(-1,1) # for the purpose of feasibility check, no need to normalize c vector
    A_ub = io.tensor(A)
    b_ub = io.tensor(b).view(-1,1)
    # solve the generated linear program min c'x st. Ax <= b, with no bounds on variables unless specified by A, b
    sol  = io.linprog (c_vector, A_ub, b_ub, eps = 1e-7).detach().numpy()
    if sol.size>0:
        return True, sol
    elif sol.size ==0:
        return False, sol  
    
# given a point, and feasible region, check if the point is feasible
def feasibility(point, A_ub, b_ub, tolerance= 6):
    m,n = A_ub.shape
    feasibility_ub = np.round(A_ub @ point.reshape((n,1)) - b_ub.reshape((m,1)), tolerance)
    if (feasibility_ub <=0).all():
        return True
    else: 
        return False

In [ ]:
def record_PLP(filename, LP, weights, w_A_coefficient, u_data, x_target, weights_corrupt ):
    c, A_ub, b_ub = as_numpy(*LP)
    m,n = A_ub.shape
    w_c, w_A, w_b,  = as_numpy(*weights)
    w_c_Corrupt, w_A_Corrupt, w_b_Corrupt = as_numpy(*weights_corrupt)
    
    u_lb, u_ub, u_train = u_data 
    u_range = np.array([u_lb, u_ub]).reshape((1,2))
        
    with open(filename, "w") as PLP_file:
        np.savetxt(PLP_file, np.array(A_ub.shape).ravel(), fmt="%d")
        PLP_file.write("\n")
        
        PLP_file.write("A_ub\n")
        np.savetxt(PLP_file, A_ub, fmt="%.6f")
        PLP_file.write("w_A\n")
        np.savetxt(PLP_file, w_A.reshape((1,2)), fmt="%.6f")
        np.savetxt(PLP_file, w_A_coefficient.reshape((1, m)), fmt="%.6f")

        PLP_file.write("\n")        
        PLP_file.write("b_ub\n")
        np.savetxt(PLP_file, b_ub.reshape((1,m)), fmt="%.6f")       
        PLP_file.write("w_b\n")
        np.savetxt(PLP_file, w_b.reshape((1,2)), fmt="%.6f")
        
        PLP_file.write("\n")  
        PLP_file.write("c_true\n")
        np.savetxt(PLP_file, c.reshape((1,n)), fmt="%.6f")        
        PLP_file.write("w_c\n")
        np.savetxt(PLP_file, w_c.reshape((1,2)), fmt="%.6f")
        
        PLP_file.write("\n")  
        PLP_file.write("u_train\n")
        np.savetxt(PLP_file, u_range, fmt="%.6f") 
        np.savetxt(PLP_file, u_train.reshape((1,len(u_train))), fmt="%.6f") 
        
        PLP_file.write("\n")      
        PLP_file.write("x_target \n") 
        np.savetxt(PLP_file, np.array(x_target), fmt="%.6f")
        PLP_file.write("\n")        
        PLP_file.write("w_A_Corrupt\n")
        np.savetxt(PLP_file, w_A_Corrupt, fmt="%.6f")

        PLP_file.write("\n")        
    
        PLP_file.write("w_b_Corrupt\n")
        np.savetxt(PLP_file, w_b_Corrupt, fmt="%.6f")
        
        PLP_file.write("\n")      
        PLP_file.write("w_c_Corrupt\n")
        np.savetxt(PLP_file, w_c_Corrupt, fmt="%.6f")

# read txt file to retrive all related data for each PLP instance
def read_PLP(filename):
    lines=[]
    with open(filename, "rt") as fp:
        for line in fp:
            lines.append(line.rstrip('\n')) 
    
    m = int(lines[0])
    n = int(lines[1])
    
    print(m,n)
    
    temp = [lines[i+1:i+m+1] for i in range(len(lines)) if "A_ub" in lines[i]]
    A_ub = []
    for i in range(len(temp[0])):
        te = np.fromstring(temp[0][i], dtype=float, sep=' ')
        A_ub.append(te)
    A_ub = np.array(A_ub)
    
    temp = [lines[i+1:i+3] for i in range(len(lines)) if "w_A" in lines[i]
           and "Corrupt" not in lines[i]]
    w_A = []
    te = np.fromstring(temp[0][0], dtype=float, sep=' ')
    w_A = np.array(te)
    te = np.fromstring(temp[0][1], dtype=float, sep=' ')
    w_A_coefficient =np.array(te).reshape((1,m))
    
    temp = [lines[i+1] for i in range(len(lines)) if "b_ub" in lines[i]]
    te = np.fromstring(temp[0], dtype=float, sep=' ')
    b_ub = np.array(te).reshape((m,1))
    
    temp = [lines[i+1] for i in range(len(lines)) if "w_b" in lines[i]
           and "Corrupt" not in lines[i]]
    te = np.fromstring(temp[0], dtype=float, sep=' ')
    w_b = np.array(te)
    
    temp = [lines[i+1] for i in range(len(lines)) if "c_true" in lines[i]]
    c_true = np.fromstring(temp[0], dtype=float, sep=' ').reshape((-1,1))
    
    temp = [lines[i+1] for i in range(len(lines)) if "w_c" in lines[i] 
            and "Corrupt" not in lines[i]]
    w_c = []
    te = np.fromstring(temp[0], dtype=float, sep=' ')
    w_c = np.array(te)

    temp = [lines[i+2] for i in range(len(lines)) if "u_train" in lines[i]]
    u = []
    for i in range(len(temp[0])):
        te = np.fromstring(temp[0], dtype=float, sep=' ')
    u = te.reshape((len(te),1))
    
    temp = [lines[i+1:i+len(u)+1] for i in range(len(lines)) if "x_target" in lines[i]]
    x_target = []
    for i in range(len(temp[0])):
        te = np.fromstring(temp[0][i], dtype=float, sep=' ')
        x_target.append(te)
    x_target = np.array(x_target).reshape((len(u),n))
    
    
    temp = [lines[i+1] for i in range(len(lines)) if "w_A_Corrupt" in lines[i]]
    te = np.fromstring(temp[0], dtype=float, sep=' ')
    w_A_Corrupt = np.array(te)    
    
    temp = [lines[i+1] for i in range(len(lines)) if "w_b_Corrupt" in lines[i]]
    te = np.fromstring(temp[0], dtype=float, sep=' ')
    w_b_Corrupt = np.array(te)
    
    temp = [lines[i+1] for i in range(len(lines)) if "w_c_Corrupt" in lines[i]]
    te = np.fromstring(temp[0], dtype=float, sep=' ')
    w_c_Corrupt = np.array(te)
    
    return (c_true, A_ub, b_ub), (w_c, w_A, w_b), (w_c_Corrupt, w_A_Corrupt, w_A_coefficient, w_b_Corrupt), x_target, u


## Define ParametricLP class

#### See "parametric.py" for more details

In [ ]:
class ParametricLP(io.ParametricLP):

    #initialize parameters
    def __init__(self, PLP, weights, w_A_coefficient):
        super().__init__(weights)
        
        c_base, A_base, b_base = PLP
        self.c_base = io.tensor(c_base).view(-1,1)
        self.A_base = io.tensor(A_base)
        self.b_base = io.tensor(b_base).view(-1,1)
        
        self.w_A_coefficient = w_A_coefficient.ravel()
        
        self.weights = as_tensor(weights)
        self.weights.requires_grad_()
                
    def zero_grads(self):
        if self.weights.grad is not None:
            self.weights.grad.detach_()
            self.weights.grad.data.zero_()
            
    # unpack weights, weigths = [w_c, w_A, w_b]      
    def unpack_weights(self, weights):
        m, n = self.A_base.shape
        w_c = weights[:2].view((1,2))
        w_A = weights[2:4].view((1,2))
        w_b = weights[4:].view((1,2))
        return w_c, w_A, w_b
    
    # genearte corresponding PLP using the given u's and weights
    def generate(self, u, weights):        
        m,n = self.A_base.shape
        
        w_c, w_A, w_b = self.unpack_weights(weights)
        
        # add linear function to a random element of each row of A
        A_PLP = torch.zeros((m,n),dtype=torch.double)
        for j in range(m):
            A_PLP[j, self.w_A_coefficient[j]] = w_A[0,0]+w_A[0,1]*u
        A_PLP = torch.add(A_PLP, self.A_base)
        
        # add linear function to every element of b and c
        b_PLP = torch.zeros((m,1),dtype=torch.double)
        for j in range(m):
            b_PLP[j] = w_b[0,0] + w_b[0,1]*u
        b_PLP = torch.add (b_PLP, self.b_base)

        c_PLP = torch.zeros((n,1),dtype=torch.double)
        for i in range(n):
            c_PLP[i] = w_c[0,0] + w_c[0,1]*u
        c_PLP = torch.add (c_PLP, self.c_base) 
        
        return c_PLP, A_PLP, b_PLP, None, None
    
    #test the PLP_true to find upperbound and lowerbound for u
    def find_uRange(self):
        u_range = []
        step, u_lb, u = 0, 0, -0.05
        print("\n||| find min(u) |||")
        while u_lb>-1.0:
            try:
                c_PLP, A_PLP, b_PLP, _, _ = self.generate(u, self.weights)
                LP_test_flag, sol = solve_lp(c_PLP, A_PLP, b_PLP)
                if LP_test_flag and max(abs(sol))<20:
                    if u<= u_lb: # decrease u_lb if PLP(ui) is feasible and has more than 2 active constraints
                        u_lb = u
                    print("||| step %d, GOOD PLP instance with u =%.2f"% (step,u))
                else:
                    print("||| step %d, BAD PLP instance with u =%.2f"% (step,u))
                    break
            except io.InfeasibleConstraintsError:
                # linprog_feasible failure
                print("||| step %d, PLP(u = %.2f) raise a InfeasibleConstraintsError"% (step,u))
                break
            except io.UnboundedConstraintsError:
                # linprog_feasible failure
                print("||| step %d, PLP(u = %.2f) raise a UnboundedConstraintsError"% (step,u))
                break
            except io.MatrixInverseError:
                # linprog_feasible failure
                print("||| step %d, PLP(u = %.2f) raise a MatrixInverseError"% (step,u))
                break        
            u -=0.05
            step+=1
        
        step, u_ub, u = 0, 0, 0.05
        print("\n||| find max(u) |||")
        while u_ub<1.0:
            try:
                c_PLP, A_PLP, b_PLP, _, _ = self.generate(u,self.weights)
                LP_test_flag, sol = solve_lp(c_PLP, A_PLP, b_PLP)
                if LP_test_flag and max(abs(sol))<20:
                    if u>= u_ub:# increase u_ub if PLP(ui) is feasible and has more than 2 active constraints
                        u_ub = u
                    print("||| step %d, GOOD PLP instance with u =%.2f"% (step,u))
                else:
                    print("||| step %d, BAD PLP instance with u =%.2f"% (step,u))
                    break
            except io.InfeasibleConstraintsError:
                # linprog_feasible failure
                print("||| step %d, PLP(%.2f) raise a InfeasibleConstraintsError"% (step,u))
                break
            except io.UnboundedConstraintsError:
                # linprog_feasible failure
                print("||| step %d, PLP(%.2f) raise a UnboundedConstraintsError"% (step,u))
                break
            except io.MatrixInverseError:
                # linprog_feasible failure
                print("||| step %d, PLP(%.2f) raise a MatrixInverseError"% (step,u))
                break
            u +=0.05
            step+=1
        print("||| bound for u = [%.2f, %.2f] |||\n"%(u_lb, u_ub))
        return u_lb, u_ub
        
    # generate the complete PLP and record all related data using record_PLP function
    def generate_PLP(self, num_u):
        u_bound = self.find_uRange()
        u_lb, u_ub = u_bound
        m,n = self.A_base.shape            
        print("\n===== Test new True_PLP over %d u_Train====="%num_u)
        # given the true PLP and u = [lb, ub] from test_uRange, generate num_u observations and their corresponding LP
        x_target = []
        print("u_range [%.2f, %.2f]"%(u_lb, u_ub))
        # generate equaly spaced u_train, abs(u_lb) and abs(u_ub) > 1.0, might give wired feasible region, 
        # u~[-1,1] tend to give PLP within a reasonable range with similar shape
        u_train = np.round(np.linspace( max(-1,u_lb), min(1,u_ub), num_u),3) 
        print("u_train", u_train)
        u_data = (u_lb, u_ub, u_train)
        
        # compute x_target for training
        for ind_u, ui in enumerate (u_train):
            temp = io.linprog(*self.generate(ui, self.weights), eps=0.0000001).detach()
            if max(abs(temp))<20: # make sure the x_target of u_train is not located too far away from the origin
                x_target.append(temp.numpy().ravel())            
            else:
                return False, None, None    
        
        if n<=2:
            fig = self.plot_PLP(u_train, x_target)
      
            return True, np.array(x_target), u_data, fig
        else:
            return True, np.array(x_target), u_data
    
    def plot_PLP(self, u_train,x_target):
        alpha = 0.25
        fig = plt.figure(dpi=100, figsize=(5, 5))
        ax = fig.add_subplot(111)
        ax.set_xlim(-5, 5)
        ax.set_ylim(-5, 5)
        ax.set_aspect('equal', 'box')
        fig.tight_layout()
        
        for ind_u, ui in enumerate (u_train):
            alpha_i = alpha + (1-alpha)*(ind_u/(len(u_train)-1) if len(u_train) > 1 else 1)
            c, A_ub, b_ub, _, _ = self.generate(ui, self.weights)
            plot_flag = iop.plot_linear_program(c,A_ub, b_ub, color='k', alpha= alpha_i,ax = ax)
            
        
        iop.plot_targets(np.array(x_target), marker='o', color='k', 
                     markerfacecolor='white', markersize=10.0, 
                     markeredgewidth=1.5, alpha=0.25)
        
        return plt
        
    # given the a set of u_train's obtained above from PLP(true_weights), 
    # corrupt the true_weights so that all PLP(corrupt_weights, u_train) still corresponds to feasible LPs
    def corrupt_PLP(self, u_train, sigma):
        print("\n\n ====== corrupting PLP_true ======")
        
        # corrupt the True Weights
        def corrupt_weights(sigma):
            w_c, w_A, w_b = self.unpack_weights(self.weights)
            m,_ = w_A.shape
            n,_ = w_c.shape  

            w_A_noise = np.random.uniform(sigma*-1,sigma, (1,2))
            w_A_corrupt = np.round( np.add(w_A.detach().numpy(), w_A_noise), 6)

            w_b_noise = np.random.uniform(sigma*-1,sigma, (1,2))
            w_b_corrupt = np.round( np.add(w_b.detach().numpy(), w_b_noise), 6)

            w_c_noise = np.random.uniform(sigma*-1,sigma, (1,2))
            w_c_corrupt = np.round( np.add(w_c.detach().numpy(), w_c_noise), 6)
            
            weights = np.append(np.append( w_c_corrupt.reshape(-1), 
                                          w_A_corrupt.reshape(-1)  ), 
                                w_b_corrupt.reshape(-1))

            return io.tensor(weights)
        
        m,n = self.A_base.shape
        flag = False

        while flag == False: # regenerate new corrupted weights if any of the corrupt weights gives infeasible LPs 
            print("\n===== generate new corrupted weights =====")
            weights_corrupt = corrupt_weights(sigma)

            x_init = []
            i=0
            feasibleLP = True
            while i <len(u_train): #check all ui in u_train
                print("***** Observation %d, ui = %.3f"%(i, u_train[i]))
                c_Corrupt, A_Corrupt, b_Corrupt, _, _ = self.generate(u_train[i], weights_corrupt)
                try:
                    # solve the forward problem with PLP_corrupt(corrupt_weights, u_train[i])
                    x_init.append( io.linprog(io.tensor(c_Corrupt).view(-1,1),
                                            io.tensor(A_Corrupt),
                                            io.tensor(b_Corrupt).view(-1,1),
                                            eps=0.0000001).detach().numpy().ravel())

                    if max(abs(x_init[i]))>20:
                        print("||| PLP with corrupt weights has large x_init")
                        break
                    else:
                        print("++++++++++ find a feasible x_init: ", x_init[i])
                        if i == len(u_train)-1: # leave the while loop, if all ui in u_train are checked
                            flag = True
                        i+=1
                except io.InfeasibleConstraintsError:
                    print("||| PLP with corrupt weights raise a InfeasibleConstraintsError"% u_train[i])
                    break
                except io.UnboundedConstraintsError:
                    print("||| PLP with corrupt weights raise a UnboundedConstraintsError"% u_train[i])
                    break
                except io.MatrixInverseError:
                    print("||| PLP with corrupt weights raise a MatrixInverseError"% u_train[i])
                    break
        print("corrupted weights = \n", (*weights_corrupt.numpy()))

        return weights_corrupt
    


## I. Generate PLP instances

run the following cell to generate complete test instances for Parametric LP case:
#### Note, for PLP, we only generate instances with two different sizes: 2D with 8 constraints and 10D with 36 constraints, 50 instances each.

In [ ]:
seed_random(0)
sigma = 0.2 # for generating the true weights
corrupt_sigma = 0.2 # for corrupting the weights
num_u = 20
num_lp = 50

nVar_range = [2,10]
nCons_range = [[8],
               [36]]

for i, nVar  in enumerate (nVar_range):
    for nCons in nCons_range[i]:
        print(nVar, nCons)
        for ind_lp in range(num_lp):
            LP_filename = PATH + "Deep_Inv_Opt/figure3_i/n=%d,m=%d/%svar_LP%s.txt"%(nVar, nCons, str(nVar).zfill(2), str(ind_lp+1).zfill(2)) 
            # reading from 3a instances
            A_ub, b_ub, _, c = Read_3aLP(LP_filename)
            print("c\n",c)
            print("A\n",A_ub)  
            print("b\n",b_ub.ravel())
        
            flag = False
            while flag == False:
                #Generate true_PLP, w0 = 0, and w1 = normal(0, sigma)
                w_c = np.round( np.column_stack( (np.zeros((1,1)), np.random.normal(0, sigma, 1)) ), 6)
                w_A = np.round( np.column_stack( (np.zeros((1,1)), np.random.normal(0, sigma, 1)) ), 6)
                w_b = np.round( np.column_stack( (np.zeros((1,1)), np.random.normal(0, sigma, 1)) ), 6)
                w_A_coefficient = np.random.randint(0, nVar, nCons)

                print("w_c\n",w_c)
                print("w_A_coefficient\n", w_A_coefficient.ravel())
                print("w_A\n",w_A)  
                print("w_b\n",w_b)

                print()
                true_weights = np.append(np.append( w_c.reshape(-1), w_A.reshape(-1)  ), w_b.reshape(-1))
                base_LP = (c, A_ub, b_ub)

                PLP_true = ParametricLP(base_LP, true_weights, w_A_coefficient)
                print()

                print("==== Generate a set of observations (u's) ====")
                if nVar <=2:
                    flag, x_target, u_data, plt = PLP_true.generate_PLP(num_u = num_u)
                else: 
                    flag, x_target, u_data = PLP_true.generate_PLP(num_u = num_u)
                if flag == True:
                    print("generate a good PLP_true with 20 u's")
                    print("x_target of PLP_true", x_target)
                    
                    directory = PATH + "Deep_Inv_Opt/figure4_plp/n=%d,m=%d/"%(nVar, nCons)
                    if not os.path.exists(directory):
                        os.makedirs(directory)
                    
                    if nVar <= 2:
                        figname = directory + "%svar_LP%s_True"%(str(nVar).zfill(2), str(ind_lp+1).zfill(2)) 
                        plt.savefig(figname, bbox_inches='tight', dpi=100)
                    
            u_lb, u_ub, u_train = u_data

            weights_corrupt = PLP_true.corrupt_PLP (u_train, sigma = corrupt_sigma)
            
            PLP_filename = directory + "%svar_LP%s.txt"%(str(nVar).zfill(2), str(ind_lp+1).zfill(2)) 
            # record all relavent data for the PLP
            record_PLP(PLP_filename, (PLP_true.c_base, PLP_true.A_base, PLP_true.b_base), 
                       (PLP_true.unpack_weights(PLP_true.weights)), 
                       PLP_true.w_A_coefficient, 
                       u_data, x_target,
                       (PLP_true.unpack_weights(weights_corrupt)) ) 


## II. Experiment - Learn A, b and c jointly for Parametric LP

#### Sample hyperparameters

In [ ]:
_RANGE_CONS = [[4,  8,  16],
               [20, 36, 80]]
_RANGE_VAR = [2, 10]

_RANGE_MU= [1.5, 2, 5, 10, 20]
_RANGE_T0 = [0.5, 1, 5, 10]
_RANGE_LR_Ab = [1, 10]
_LR_C_AB_RATIO = [100, 1, 0.01]
_RANGE_EPS = [1e-5]
_LOSS_FUNCTION = ["MSE"]
_HYPERPARAM = (_RANGE_MU, _RANGE_T0, _RANGE_EPS, _RANGE_LR_Ab,_LR_C_AB_RATIO)


def sample_hyperparam(filename, num_set = 20, print_hyperparam = True):
    hyperParam = [[]]*num_set
    for i in range(num_set):
        mu = _RANGE_MU[np.random.randint(0, len(_RANGE_MU))]
        t0 = _RANGE_T0[np.random.randint(0, len(_RANGE_T0))]
        eps = _RANGE_EPS[np.random.randint(0, len(_RANGE_EPS))]
        
        lr_Ab = _RANGE_LR_Ab[np.random.randint(0, len(_RANGE_LR_Ab))]
        lr_c = lr_Ab*_LR_C_AB_RATIO[np.random.randint(0, len(_LR_C_AB_RATIO))]
        hyperParam[i] = (mu, t0, eps, lr_c, lr_Ab)
        if print_hyperparam:
            print("hyperparam set %d"%(i))
            print(hyperParam[i])
        
    with open(filename, 'w') as file:
        for j in range(len(hyperParam)):
            file.write("hyperParam%s: "%(str(j+1).zfill(2)))
            np.savetxt(file, np.mat(hyperParam[j]).ravel(), fmt='%.6f')
        file.write("\n")

def read_hyperParam(filename, num_hyper = 20):
    with open(filename, 'rt') as file:
        lines=file.readlines()
        temp = [lines[i] for i in range(len(lines)) if "hyperParam" in lines[i]]
        hyperParam = [[]]*num_hyper
        for i in range(len(temp)):
            te = temp[i]
            te = te[14:]
            te = te[:-2]
            te = te.split(' ')
            hyperParam[i] = [float(j) for j in te]
    return hyperParam

print("==== Sample Hyperparameters values for Random Hyperparameter Search====")
seed_random()
filename = PATH + "Deep_Inv_Opt/Figure4_plp/Hyper_Param.txt"
sample_hyperparam(filename)
read_hyperParam(filename)


### run the following cell for experiment of learning Parametric LPs using Deep_Inv_Opt package

#### Please Note, finishing the entire experiments (i.e., 2 * 50 instances for Training with MSE) might take 1-2 days. You can see our paper for the experiment results directly.

In [ ]:
def exp_PLP(nVar, nCons, ind_lp, hyperParam,num_u):
    PLP_filename = PATH + "Deep_Inv_Opt/figure4_plp/n=%d,m=%d/%svar_LP%s.txt"%(nVar, nCons, str(nVar).zfill(2), str(ind_lp+1).zfill(2)) 
    print(filename)
    base_PLP, weights_true, weights_corrupt, x_target, u = read_PLP(PLP_filename)
    c, A_ub, b_ub = base_PLP
    w_c, w_A, w_b = weights_true
    w_c_corrupt, w_A_corrupt, w_A_coefficient, w_b_corrupt = weights_corrupt

    w_A_coefficient = w_A_coefficient.astype(int)

    print("w_c\n",w_c)
    print("w_A_coefficient\n", w_A_coefficient.ravel())
    print("w_A\n",w_A)  
    print("w_b\n",w_b)

    print("w_c_corrupt\n",w_c_corrupt)
    print("w_A_corrupt\n",w_A_corrupt)  
    print("w_b_corrupt\n",w_b_corrupt)

    print()

    #initialize all data matrix
    record_loss=9999
    
    corrupt_weights = np.append(np.append( w_c_corrupt.reshape(-1), w_A_corrupt.reshape(-1) ), w_b_corrupt.reshape(-1))

    result_file = PATH+"Deep_Inv_Opt/figure4_plp/FinalExp _%svar_%scon_%s.txt"%(str(nVar).zfill(2),str(nCons).zfill(2), "MSE")
    with open(result_file, 'a') as file:
        # print LP size, m - # constrains, n - # # var
        file.write( "LP%s, %svar %scon:\n"
                   %(str(ind_lp+1).zfill(2),str(nVar).zfill(2),str(nCons).zfill(2)))
        file.write( "x_target \n" ) 
        np.savetxt(file, x_target.reshape(num_u,nVar), fmt='%.6f')
        file.write("weights_init: ")
        np.savetxt(file, corrupt_weights.reshape(1,len(corrupt_weights)), fmt='%.6f')
    
    hyperparam_flag=False
    for ind_search in range(len(hyperParam)):
        print("---- EXP %d / %d: "%(ind_search+1, len(hyperParam)))

        if hyperparam_flag == False:
            print("Have not found a hyperparam set give err < 1e-4")
            # initialize hyper parameters

            # assign different learning rate
            # to use this, one must re-write the sample_hyperparam() function in the previous cell
            mu, t0, eps, lr_c, lr_Ab = hyperParam[ind_search]
            Globallr = [1 for i in range(len(corrupt_weights))]
            for ind_lr in range(len(Globallr)):
                if ind_lr<2:
                    Globallr[ind_lr] = lr_c
                else:
                    Globallr[ind_lr] = lr_Ab
            #initialize the corrupt_weights
            corrupt_weights = np.append(np.append( w_c_corrupt.reshape(-1), w_A_corrupt.reshape(-1) ), w_b_corrupt.reshape(-1))

            # comment the following lines to disable some printing along problem solving
            print("---- corrupt_weights",corrupt_weights)
            print("---- x_target", x_target)
            print("---- learning rate",Globallr)
            print("loss func MSE, HyperParam %d = (mu:%.1f, t0:%.1f, eps:%.5f)"
                  %(ind_search+1, mu, t0, eps))
            
            base_LP = (c, A_ub, b_ub)
            PLP = ParametricLP (base_LP, corrupt_weights, w_A_coefficient)
            u_train = io.tensor(u)
            x_target = io.tensor(x_target)
            start_time = time.time()    
            max_steps = 200
            f_learned, err_final, x_final = io.inverse_parametric_linprog(u_train, x_target, PLP, lr=Globallr, 
                                                                          max_steps=max_steps,
                                                                          eps = eps,
                                                                          eps_decay=False, # change eps_callback to using eps_decay
                                                                          callback=io.inverse_parametric_linprog_step_printer(),
                                                                          solver=io.custom_linprog(t0 = t0, mu=mu))
            
            runtime = time.time() - start_time # compute runtime

            w = f_learned.weights.data.detach().numpy()
            x = x_final.detach().numpy().ravel()
            l = err_final.detach().numpy().ravel()
            t = runtime

            # for each hyperparam search, record the experimen data:
            with open(result_file, 'a') as file:
                    # values of hyperparameters, final error, and runtime
                    if len(hyperParam[0]) == 5:
                        temp = (mu, t0, eps, lr_c, lr_Ab, l[0], t)
                    string = ("hyperparam"+str(ind_search+1).zfill(2)+" "+ str(temp) )
                    file.write(str(string))
                    file.write("\n")
                    # record final_weights
                    file.write("weights_final ")
                    np.savetxt(file, w.reshape((1, len(w))), fmt='%.6f') 
                    # record x_final
                    file.write("x_final ")
                    np.savetxt(file, x.reshape(num_u,nVar), fmt='%.6f') 

            print("final_err", err_final.detach().numpy().ravel())

            #tracking the best-found error from hyperparam search, this is not recorded in data file
            if err_final.detach().numpy() <= record_loss:
                print("--- :) find better loss \n")
                record_loss = err_final.detach().numpy().ravel()
                best_hyperparam = ind_search
            else:
                print("--- :( no better loss found\n")
            if record_loss<1e-4:
                hyperparam_flag =True
        elif hyperparam_flag == True:
            print("Found a hyperparam set give err < 1e-4")
            print("skip the rest of hyperparam search, and record dummy experiment data")
            
             # for each hyperparam search, record the experimen data:              
            with open(result_file, 'a') as file:
                    # values of hyperparameters, final error, and runtime
                    mu, t0, eps, lr_c, lr_Ab = hyperParam[ind_search]
                    temp = (mu, t0, eps, lr_c, lr_Ab, 9999, 9999)
                    string = ("hyperparam"+str(ind_search+1).zfill(2)+" "+ str(temp) )
                    file.write(str(string))
                    file.write("\n")
                    # record final_weights
                    file.write("weights_final ")
                    np.savetxt(file, np.zeros((1, len(w))), fmt='%.2f') 
                    # record x_final
                    file.write("x_final\n")
                    np.savetxt(file, np.zeros((num_u,nVar)), fmt='%.2f') 


filename = PATH + "Deep_Inv_Opt/Figure4_plp/Hyper_Param.txt"
hyperParam = read_hyperParam(filename)
num_u = 20  # num of training set
num_lp=1

nVar_range = [2,10]
nCons_range = [[8],[36]]

for i, nVar in enumerate(nVar_range):
    for nCons in nCons_range[i]:
        for ind_lp in range(num_lp):    
            exp_PLP(nVar, nCons, ind_lp, hyperParam, num_u)
   

## III. Data Analysis

### Collect Best Training Error from experiment result data (obtained in the previous cell)


In [ ]:
# read experiment results of hyperparameter search 
def read_finalresult(num_lp, nCons, nVar, loss="MSE"):
    # read x_target
    x_target = []
    result = []
    weights_final = []
    x_final = []

    with open(PATH + "Deep_Inv_Opt/figure4_plp/FinalExp _%svar_%scon_%s.txt"
              %(str(nVar).zfill(2),str(nCons).zfill(2), loss), 'rt') as file:
        lines=file.readlines()
    temp = [lines[i+1:i+1+5] for i in range(len(lines)) if "x_target" in lines[i]]

    for i in range(num_lp):
        for j in range(len(temp[i])):
            te = temp[i][j]
            te = te[:-2]
            te = te.split(' ')
            te = [float(d) for d in te]
            x_target.append( te)
    x_target = np.array(x_target)

    temp = [lines[i] for i in range(len(lines)) if "hyperparam" in lines[i]]    
    for i in range(len(temp)):
        te = temp[i]
        te = te[14:]
        te = te[:-2]
        te = te.split(',')
        te = [float(d) for d in te]
        result.append(te)
    result= np.array(result)
    # find the index of hyperparameter search with the best error value
    best_ind = []
    size_hyperparam = 20
    for i in range(num_lp):
        best_ind.append(int ( i*size_hyperparam + np.argmin(result[i*size_hyperparam:(i+1)*size_hyperparam,-2])))
    best_ind = np.array(best_ind)

    # read c_final for every trail
    temp = [lines[i] for i in range(len(lines)) if "weights_final" in lines[i]]
    for i in range(len(temp)):
        te = temp[i]
        te = te[14:]                
        te = te.split(' ')
        te = [float(d) for d in te]
        weights_final.append(te)
    weights_final= np.array(weights_final)


    return result, weights_final, best_ind, x_final



def TrainErr(nVar, nCons):
    trainig_error=[]
    result, weights_final, best_ind, x_final = read_finalresult(num_lp, nCons, nVar, loss="MSE")
    trainig_error = result[best_ind,-2]
    Train_MSE_filename = PATH + "Deep_Inv_Opt/figure4_plp/n=%d,m=%d/Train_MSE.txt"%(nVar, nCons) 
    with open (Train_MSE_filename,"a") as file:
        file.write("Best Training Error\n")
        np.savetxt(file, np.array(trainig_error), fmt="%.3e") 

        
num_lp=50
nVar_range = [2,10]
nCons_range = [[8],[36]]

for i, nVar in enumerate(nVar_range):
    for nCons in nCons_range[i]:
        TrainErr(nVar, nCons)

### Compute Initial Error

In [ ]:
def InitialErr(nVar, nCons, ind_lp):
    
    filename = PATH + "Deep_Inv_Opt/figure4_plp/n=%d,m=%d/%svar_LP%s.txt"%(nVar, nCons, str(nVar).zfill(2), str(ind_lp+1).zfill(2)) 
    print(filename)
    base_PLP, weights_true, weights_corrupt, x_target, u = read_PLP(filename)
    c, A_ub, b_ub = base_PLP
    w_c, w_A, w_b = weights_true
    w_c_corrupt, w_A_corrupt, w_A_coefficient, w_b_corrupt = weights_corrupt  
    w_A_coefficient = w_A_coefficient.astype(int)
    u_train = u

    true_weights = np.append(np.append( w_c.reshape(-1), w_A.reshape(-1) ), w_b.reshape(-1))

    corrupt_weights = np.append(np.append( w_c_corrupt.reshape(-1), w_A_corrupt.reshape(-1) ), w_b_corrupt.reshape(-1))

    base_LP = (c, A_ub, b_ub)

    # initialize the corrupt_weights
    PLP = ParametricLP (base_LP, corrupt_weights, w_A_coefficient)
    
    u_train = io.tensor(u)
    print("u_train",u_train.t().numpy())
    
    x_target = io.tensor(x_target)

    #compute x_init
    x_init = torch.cat([io.linprog(*PLP.generate(ui, io.tensor(corrupt_weights)), eps=0.0000001).detach().t() for ui in u_train])
    print("\n======   x_init   ======\n",x_init.detach().numpy())
    print("\n======   x_target   ======\n", x_target.detach().numpy())
    
    x_MSE_init=[]
    for ind_u, ui in enumerate (u_train):
        xi_MSE_init = torch.sum(io.squared_error(None, x_target[ind_u], x_init[ind_u]), 0)

        x_MSE_init.append(xi_MSE_init.detach().numpy())
    print("\n===== squared error of all u_train =====\n",np.array(x_MSE_init))         

    
    return np.array(x_MSE_init)
            
            
alpha = 0.25
num_lp=50
nVar_range = [2,10]
nCons_range = [[8],[36]]

for i, nVar in enumerate(nVar_range):
    for nCons in nCons_range[i]:
        for ind_lp in  range(num_lp):

            MSE_init = InitialErr(nVar, nCons, ind_lp)     
            print(MSE_init)
            init_MSE_filename = PATH + "Deep_Inv_Opt/figure4_plp/n=%d,m=%d/Init_MSE.txt"%(nVar, nCons) 
            with open (init_MSE_filename,"a") as file:
                file.write("LP%s\n"%str(ind_lp+1).zfill(2))
                np.savetxt(file, np.array(MSE_init).reshape((1,num_hyper)), fmt="%.6f") 



### Computer Test Error
Generate a set of new u values, substitute to the trained parametric LPs, i.e., PLP($c_{train}, A_{train}, b_{train}, u_{test}$), and compute the validation error

In [ ]:
def test_error(nVar, nCons, ind_lp, learn_weights,num_u, trainig_error):
    filename = PATH + "Deep_Inv_Opt/figure4_plp/n=%d,m=%d/%svar_LP%s.txt"%(nVar, nCons, str(nVar).zfill(2), str(ind_lp+1).zfill(2)) 
    base_PLP, weights_true, weights_corrupt, _, u = read_PLP(filename)
    c, A_ub, b_ub = base_PLP
    print("base_c",c)
    w_c, w_A, w_b = weights_true
    print("true_wc", w_c)
    print("learned_c", learn_weights[:2])
    _, _, w_A_coefficient, _ = weights_corrupt
    m,n = A_ub.shape
    w_A_coefficient = w_A_coefficient.astype(int)
    print(filename)

    print("u_train",u.ravel())
    u_lb, u_ub = u[0], u[-1]
    u_train = u

    base_LP = (c, A_ub, b_ub)
    true_weights = io.tensor(np.append(np.append( w_c.reshape(-1), 
                                       w_A.reshape(-1) ), 
                             w_b.reshape(-1)) )
    print("true_weights", true_weights)

    learn_weights = io.tensor(learn_weights.ravel())
    print("learn_weights", learn_weights)
    
    PLP_base = ParametricLP (base_LP, learn_weights, w_A_coefficient)

    # given the true PLP and u = [lb, ub] from test_uRange, generate 4 observations and their corresponding LP
    print("u_range", u_lb, u_ub)
    u_test = np.zeros(num_u)

    x_MSE=[]
    x_true_test=[]
    x_learn_test=[]
    u_test = np.random.uniform(u_lb, u_ub, num_u)
    for i in range(len(u_test)):
        while u_test[i] in u_train:
            u_test[i] = np.random.uniform(max(-1,u_lb), min(1,u_ub), 1)
    u_test.sort()
    print("u_test",u_test)

    for ind_u in range(num_u):

        print("====== new u_test", u_test[ind_u])

        xi_true_tests = io.linprog(*PLP_base.generate(u_test[ind_u], true_weights), eps=0.0000001).detach().t()

        xi_learn_tests = io.linprog(*PLP_base.generate(u_test[ind_u], learn_weights), eps=0.0000001).detach().t()
        print("x_learned_test", xi_true_tests.numpy().ravel())
        print("x_true_test",xi_learn_tests.numpy().ravel())
        
        xi_MSE_test = torch.sum(io.squared_error(_, xi_true_tests.t(), xi_learn_tests.t()), 0)
        print("SE btw learned_x, and true_x", xi_MSE_test.numpy().ravel())

        x_MSE.append(xi_MSE_test)
        x_true_test.append(xi_true_tests.numpy().ravel())
        x_learn_test.append(xi_learn_tests.numpy().ravel())

    print("\n\n",x_learn_test)
    # Stack the results into matricies, for convenience
    print("\nx_MSE\n",torch.cat(x_MSE).numpy())
    MSE = torch.mean(torch.cat(x_MSE))
    u_test=np.array(u_test) 
    
    print()
    print("test_error: ", MSE.detach().numpy())     
    print("best training error",trainig_error[ind_lp])
    print("====================\n")

    return u_test, torch.cat(x_MSE).numpy()




num_hyper=20
num_u = 20
num_lp=50
nVar_range = [2,10]
nCons_range = [[8],[36]]

for i, nVar in enumerate(nVar_range):
    for nCons in nCons_range[i]:
        result, weights_final, best_ind, x_final = read_finalresult(num_lp, nCons, nVar, loss="MSE")
        trainig_error = result[best_ind,-2]
        learn_weights = weights_final[best_ind,:]
        for ind_lp in  range(num_lp):

            u_test, MSE_test = test_error(nVar, nCons, ind_lp, learn_weights[ind_lp], num_u, trainig_error)
    
            print(MSE_test)
            test_MSE_filename = PATH+"Deep_Inv_Opt/figure4_plp/n=%d,m=%d/Test_MSE.txt"%(nVar, nCons) 
            with open (test_MSE_filename,"a") as file:
                file.write("LP%s\n"%str(ind_lp+1).zfill(2))
                
                file.write("u_test\n")
                np.savetxt(file, np.array(u_test).reshape((1,num_u)), fmt="%.6f")
                
                file.write("MSE_test\n")
                np.savetxt(file, np.array(MSE_test).reshape((1,num_hyper)), fmt="%.3e") 

### Create boxplot as shown in Figure 4
read all MSE data (i.e., initla, best training and test), and create boxplots

**Note, the final boxplots might not look identical to what we presented in the paper due to the randomization in baseline LP generation.**

In [ ]:
def read_MSE(nVar, nCons):
    init_MSE_file =  PATH + "Deep_Inv_Opt/figure4_plp/n=%d,m=%d/init_MSE.txt"%(nVar, nCons) 
    train_MSE_file = PATH + "Deep_Inv_Opt/figure4_plp/n=%d,m=%d/Train_MSE.txt"%(nVar, nCons) 
    test_MSE_file =  PATH + "Deep_Inv_Opt/figure4_plp/n=%d,m=%d/Test_MSE.txt"%(nVar, nCons) 
    
    init_MSE=[]
    with open(init_MSE_file, "rt") as file:
        lines=file.readlines()
    temp = [lines[i+1] for i in range(len(lines)) if "LP" in lines[i]]
    for i in range(len(temp)):
#         te = temp[i]
#         te = te[:-2]
        te = np.fromstring(temp[i], dtype=float, sep=' ')
        init_MSE.append(te)
    init_MSE = np.array(init_MSE)
    
    train_MSE=[]
    with open(train_MSE_file, "rt") as file:
        lines=file.readlines()
    temp = lines[1:]
    for i in range(len(temp)):
#         te = temp[i]
#         te = te[:-2]
        te = np.fromstring(temp[i], dtype=float, sep=' ')
        train_MSE.append(te)
    train_MSE = np.array(train_MSE)
    
    
    test_MSE=[]
    with open(test_MSE_file, "rt") as file:
        lines=file.readlines()
    temp = [lines[i+1] for i in range(len(lines)) if "MSE_test" in lines[i]]
    for i in range(len(temp)):
#         te = temp[i]
#         te = te[:-2]
        te = np.fromstring(temp[i], dtype=float, sep=' ')
        test_MSE.append(te)
    test_MSE = np.array(test_MSE)
    
    return init_MSE, train_MSE, test_MSE      


 
    
def box_plot(nVar, nCons):
    init_MSE, train_MSE, test_MSE = read_MSE(nVar, nCons) 

    MSE_all = np.column_stack(( np.mean(init_MSE, axis = 1).reshape(num_lp,1) , 
                              np.array(train_MSE).reshape(num_lp,1), 
                              np.mean(test_MSE, axis = 1).reshape(num_lp,1) ))

    for j in range(len(MSE_all[0])):
        for i in range(len(MSE_all)):
            if MSE_all[i,j] <1e-8:
                MSE_all[i,j] = 1e-8

    _, nbox = MSE_all.shape 

    y_init = MSE_all[:, 0]
    seed_random()
    x_init = np.random.uniform(1-.15, 1+0.15, size=len(MSE_all[:, 0]))
    y_train = MSE_all[:, 1]
    seed_random()
    x_train = np.random.uniform(2-.15, 2+0.15, size=len(MSE_all[:, 1]))
    y_test = MSE_all[:, 2]
    seed_random()
    x_test = np.random.uniform(3-.15, 3+0.15, size=len(MSE_all[:, 2]))

    fig = plt.figure(figsize=(5,8))
    ax = fig.add_subplot(111)
    ax.boxplot(MSE_all,sym='')
    ax.set_yscale("log")
    ax.plot(x_init, y_init, 'go', mfc="None", alpha=0.8) 
    ax.plot(x_train, y_train, 'bo', mfc="None", alpha=0.8) 
    ax.plot(x_test, y_test, 'ro', mfc="None", alpha=0.8) 

    ax.plot([], 'go',mfc="None", alpha=0.8, label = 'Initial Err (MSE)')  
    ax.plot([], 'bo', mfc="None", alpha=0.8, label = 'Best Training Err (MSE)')
    ax.plot([], 'ro',mfc="None", alpha=0.8, label = 'Test Err(MSE)') 
    ax.set_xticks([])
    plt.ylim(0.5e-8, 1e3)

    plt.savefig(PATH+"Deep_Inv_Opt/figure4_plp/%svar%scons_complete_MSE.pdf"%(str(nVar).zfill(2), str(nCons).zfill(2)),
                frameon = True, bbox_inches='tight', dpi=100)
    plt.show()
    
nVar_range = [2,10]
nCons_range = [[8],[36]]

num_lp=50
for i, nVar in enumerate(nVar_range):
    for nCons in nCons_range[i]:
        box_plot(nVar, nCons)